In [1]:
from ultralytics import YOLO
import os
import gdown
import shutil
import yaml
import pickle

### Вспомогательные функции

In [2]:
# Загрузить разметку с гугл диска
def load_pseudo_labels(
        dataset_name, #VOC
        ann_name, # Название разметки
        ann_postfix, # Постфикс разметки
        url # Урл, чтобы скачать (google disk)
        ):
    
    save_dir = f'../data/pseudo_labels/{dataset_name}'
    os.makedirs(save_dir, exist_ok=True)
    # Загрузка разметки
    filename = f'{save_dir}/{ann_name}.zip'
    gdown.download(url, filename, quiet = True, fuzzy = True)
    !unzip -d {save_dir} {filename} 

    # Копирование в папку с датасетом
    for sample in (['train', 'test', 'val']):
        lbs_source = f'{save_dir}/{ann_name}/{sample}'
        if os.path.exists(lbs_source) is False:
            print(f'{lbs_source} does not exisist')
            continue
        lbs_dest = f'../datasets/{dataset_name}/labels/{sample}_{ann_postfix}'
        shutil.copytree(lbs_source, lbs_dest)

# Подгрузить разметку в зависимости от эксперимента
def prepare_labels(dataset_name, # Название датасета
                   train_postfix=None, # Разметка, по умолчанию оригинальная [orig, GD]
                   val_postfix=None, # Разметка, по умолчанию оригинальная [orig, GD]
                   test_postfix=None # Разметка, по умолчанию оригинальная [orig, GD]
                   ):
    # Перезаписать разметку на разметку на выбранную
    for sample, postfix in zip(['train', 'val', 'test'], [train_postfix, val_postfix, test_postfix]):
        if postfix is None:
            postfix = 'orig'

        lbs_dest = f'../datasets/{dataset_name}/labels/{sample}'
        lbs_source = f'../datasets/{dataset_name}/labels/{sample}_{postfix}'
        !rm -rf {lbs_dest}
        shutil.copytree(lbs_source, lbs_dest)

# Чтение конфигурационного файла
def read_yaml_file(file_path):
    with open(file_path, 'r') as file:
        try:
            data = yaml.safe_load(file)
            return data
        except yaml.YAMLError as e:
            print("Error reading YAML file:", e)

### Загрузка разметки

In [15]:
# Загрузить разметку с гугл диска
# load_pseudo_labels(
#         dataset_name='VOC', #VOC
#         ann_name='annotations_nms_0.7_p_0.4', # Название разметки
#         ann_postfix='GD', # Постфикс разметки
#         url='https://drive.google.com/file/d/1K9-6IppbpJTnLivh80Or4Oj5tZUgd3ej/view?usp=sharing' # Урл, чтобы скачать (google disk)
#         )


### Запуск экспериментов

In [3]:
experiments = read_yaml_file('../configs/VOC_original.yaml')
for exp in experiments:
    prepare_labels(**exp['data_config'])
    model = YOLO(exp['model'])
    results = model.train(**exp['train_config'])

    directory_old_train = str(results.save_dir)
    dir_new_train = f'{exp["saving_directory"]}_{exp["model"].split(".")[0]}_train'
    os.rename(directory_old_train, dir_new_train)

    #test
    weight_path = f'{dir_new_train}/weights/best.pt'
    model =YOLO(weight_path)
    validation_results = model.val(**exp['test_config'])
    pickle.dump(validation_results.box, file = open(f'{str(validation_results.save_dir)}/results.pickle', "wb"))

    directory_old_test = str(validation_results.save_dir)
    dir_new_test = f'{exp["saving_directory"]}_{exp["model"].split(".")[0]}_test'
    os.rename(directory_old_test, dir_new_test)


New https://pypi.org/project/ultralytics/8.1.41 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.1.35 🚀 Python-3.10.6 torch-2.2.2+cu121 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 5938MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=../datasets/VOC/VOC.yaml, epochs=3, time=None, patience=100, batch=-1, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=2, project=None, name=train13, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=No

train: Scanning /home/nikolay/Documents/Projects/Diploma/My repos/Pseudo_label_YOLO/datasets/VOC/labels/train... 2501 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2501/2501 [00:00<00:00, 2925.52it/s]

train: New cache created: /home/nikolay/Documents/Projects/Diploma/My repos/Pseudo_label_YOLO/datasets/VOC/labels/train.cache



val: Scanning /home/nikolay/Documents/Projects/Diploma/My repos/Pseudo_label_YOLO/datasets/VOC/labels/val... 2510 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2510/2510 [00:01<00:00, 2504.08it/s]


val: New cache created: /home/nikolay/Documents/Projects/Diploma/My repos/Pseudo_label_YOLO/datasets/VOC/labels/val.cache
Plotting labels to runs/detect/train13/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000417, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.00040625000000000004), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 2 dataloader workers
Logging results to runs/detect/train13
Starting training for 3 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  87%|████████▋ | 271/310 [01:15<03:00,  4.63s/it]

WARNING ⚠️ NMS time limit 2.800s exceeded


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  93%|█████████▎| 289/310 [02:00<00:43,  2.08s/it]

: 

: 

In [11]:
validation_results.box.map50

0.651476823857488

In [12]:
# metrics.box.map    # map50-95
# metrics.box.map50  # map50
# metrics.box.map75  # map75
# metrics.box.maps 
validation_results.box.map

0.4456083040486212